##Проверка гипотез

In [90]:
import pandas as pd
import numpy as np
import scipy.stats as stats

In [91]:
df = pd.read_csv('clean.csv', sep = ';')
df

,Game title,Amount of recommendations,Current price,Current discount in percent,Genre,Publisher,Rating,Names,Collection,Release,engine,Platforms,Number of Platforms
0,Counter-Strike: Global Offensive,2999135,0.00,0,Shooter,Hidden Path Entertainment,81,"Counter-Strike: Ofensiva Global, Counterstrike...",Counter-Strike,2012,Source,"Linux, PC (Microsoft Windows), PlayStation 3, ...",5
1,PUBG: BATTLEGROUNDS,1382744,29.99,0,Shooter,Bluehole Studio,75,"Battlegrounds, Player Unknown, PUBG, PLAYERUNK...",NaN,2017,Unreal Engine,"PC (Microsoft Windows), PlayStation 4, Xbox On...",4
2,Tom Clancy's Rainbow Six Siege,710454,19.99,0,Shooter,Ubisoft Montreal,74,"R6:S, R6S, R6: Siege, 彩虹六号：围攻, Regenbogen 6 Be...",Rainbow Six,2020,AnvilNext,"PC (Microsoft Windows), PlayStation 4, Xbox On...",6
3,Garry's Mod,557779,9.99,0,Simulator,Facepunch Studios,74,"GMod, 게리 모드, gmod 9, gmod",NaN,2004,Source,"Linux, PC (Microsoft Windows), Mac",3
4,Terraria,536708,9.99,0,Platform,Headup Games,81,"Terraria: Journey's End, テラリア, 테라리아 (Terraria)...",Terraria,2011,Unique,"Linux, PC (Microsoft Windows), PlayStation 3, ...",15
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2187,Yomawari: Midnight Shadows,303,19.99,0,Puzzle,NIS America,83,深夜廻,NaN,2017,Unique,"PC (Microsoft Windows), PlayStation Vita, Play...",3
2188,SiNKR,302,3.99,0,Puzzle,Robert Wahler,68,"シズめル, SiNKR",NaN,2017,Unique,"Linux, PC (Microsoft Windows), Mac, iOS, Xbox One",5
2189,Invisigun Reloaded,302,19.99,0,Arcade,Sombr Studio LLC,74,Invisigun Heroes,NaN,2019,Unique,"Linux, PC (Microsoft Windows), Mac, iOS, PlayS...",6
2190,Tropico 3: Absolute Power,302,9.99,0,Real Time Strategy (RTS),Haemimont Games,74,"Тропико 3: Абсолютная власть. Дополнение, トロピコ...",Tropico,2010,Unique,PC (Microsoft Windows),1


##Гипотеза 1

*All due respect, you got no idea what it's like to be (Hypothesis) Number One.*

Для начала я предлагаю проверить гипотезу о том, что математическое ожидание числа платформ у игры будет равняться моде этого параметра, против альтернативной, что она не равна

In [92]:
df['Number of Platforms'].mode().values[0]

1

\begin{align}
H_0: μ_{platforms} = \mathrm{1}
\end{align}
\begin{align}
H_1: μ_{platforms} \neq \mathrm{1}
\end{align}

В данном случае нам неизвестна истинная дисперсия, поэтому мы не сможем провести z-test. Поэтому выбираем t-test и проверяем на уровне значимости 5%

In [93]:
t_stat, p_value = stats.ttest_1samp(df['Number of Platforms'], 1)
if p_value < 0.05:
  print('Гипотеза отвергается')
else:
  print('Гипотеза не отвергается')

Гипотеза отвергается


В нашем случае гипотеза отвергается, следовательно, математическое ожидание числа платформ у игры не может равняться моде, то есть 1.

In [94]:
df['Number of Platforms'].mean()

3.5807481751824817

##Гипотеза 2
*Whenever I'm about to do something, I think, 'Would an idiot do that?' and if they would, I do not do that thing*

Для второй гипотезы предлагаю разбить игры на 2 группы: хорошие и плохие. За хорошие будем считать те игры, рейтинг которых не менее 75, плохие - наоборот.

In [95]:
df_good = df[df['Rating'] >= 75]
df_bad = df[df['Rating'] < 75]

Проверим гипотезу о том, что в среднем плохие и хорошие игры стоят одинаково против гипотезы о том, что хорошие игры дороже плохих, на уровне значимости 5%.

\begin{align}
H_0: μ_{good} = μ_{bad}
\end{align}
\begin{align}
H_1: μ_{good} > μ_{bad}
\end{align}

Проведём t-test для двух независимых выборок, считая, для упрощения, что их истинные дисперсии равны.

In [96]:
good_price = df_good['Current price']
bad_price = df_bad['Current price']
t_stat, p_value = stats.ttest_ind(good_price, bad_price)
t_q = stats.t.ppf(0.95, len(df) - 2)
if t_stat > t_q or t_stat < -t_q:
  print('Гипотеза отвергается')
elif t_stat > -t_q and t_stat < t_q:
  print('Гипотеза не отвергается')

Гипотеза отвергается


Гипотеза о равенстве мат. ожиданий отвергается в пользу гипотезы о том, что в среднем хорошие игры стоят дороже плохих.

##Гипотеза 3

*Sometimes In Life When You Get What You Want* (high price)*, You End Up Missing What You Left Behind* (many recommendations)

Проверим гипотезу о том, что цена игры не зависит от количества рекомендаций хи-квадрат-теста.

Посчитаем, что рекомендаций много, когда их число больше 5000. Создадим таблицу из матрицу 2х2, в первой ячейке будет количество игр, у которых много рекомендаций и цена не меньше 14.99, во второй - игры, у которых много рекомендаций и цена меньше 14.99, в третьей - игры, у которых мало рекомендаций и цена не меньше 14.99, в четвёртой - игры, у которых мало рекомендаций и цена меньше 14.99.

In [97]:
rec_price = np.array([[len(df[df['Amount of recommendations'] >= 5000][df['Current price'] >= 14.99]), len(df[df['Amount of recommendations'] >= 5000][df['Current price'] < 14.99])],
                     [len(df[df['Amount of recommendations'] < 5000][df['Current price'] >= 14.99]), len(df[df['Amount of recommendations'] < 5000][df['Current price'] < 14.99])]])

<ipython-input-97-7b8eb3b681db>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  rec_price = np.array([[len(df[df['Amount of recommendations'] >= 5000][df['Current price'] >= 14.99]), len(df[df['Amount of recommendations'] >= 5000][df['Current price'] < 14.99])],
<ipython-input-97-7b8eb3b681db>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  [len(df[df['Amount of recommendations'] < 5000][df['Current price'] >= 14.99]), len(df[df['Amount of recommendations'] < 5000][df['Current price'] < 14.99])]])


In [98]:
many_much = rec_price[0][0]/sum(rec_price[0])
lil_much = rec_price[1][0]/sum(rec_price[1])
many_much > lil_much

True

Доля игр, у которых много рекомендаций и большая цена, больше доли тех, у которых мало рекомендаций и маленькая цена

In [99]:
chi, p_value, k, _ = stats.chi2_contingency(rec_price, correction = False)
chi, p_value

(104.80437363287434, 1.3480947349534227e-24)

In [100]:
if p_value < 0.05:
  print('Гипотеза отвергается')
else:
  print('Гипотеза не отвергается')

Гипотеза отвергается


p-value меньше уровня значимости 5%, следовательно, гипотеза об однородности групп отвергается. Получается, опираясь на то, что доля игр, у которых много рекомендаций и большая цена, больше доли тех, у которых мало рекомендаций и маленькая цена, можно сказать, что цена игры может зависеть от количества рекомендаций.

##Гипотеза 4

*-I have one rule: New is always better.*

*-You can't keep changing your one rule if they're always different rules.*

*-But "New is always better" is my oldest rule, which makes it the best.*

Проверим гипотезу о том, что в среднем рейтинг игр, которые были выпущены в последние 5 лет, больше рейтинга игр, выпущенных раньше

\begin{align}
H_0: μ_{new} > μ_{bad}
\end{align}

Проведём t-test для двух независимых выборок. В этот раз для начала проверим, явлются ли дисперсии этих выборок одинаковыми с помощью теста Левена, и, если нет, проведём t-test Уэлча.

In [101]:
new = df[df['Release'] >= df['Release'].max() - 5]['Rating']
old = df[df['Release'] < df['Release'].max() - 5]['Rating']
_, p_value_1 = stats.levene(new, old)
_, p_value = stats.ttest_ind(new, old, equal_var=(p_value_1 > 0.05))
if p_value > 0.05:
  print('Гипотеза не отвергается')
else:
  print('Гипотеза отвергается')

Гипотеза не отвергается


Гипотеза не отвергается, следовательно, средний рейтинг игр выпущенных в последние 5 лет может быть больше, чем у игр, выпущенных ранее.

##Предсказание количества рекомендаций

Линейная регрессия

В качестве целевой переменной берём столбец Amount of recommendations, в качестве вспомогательных признаков - genre, current price, engine, number of platforms, rating

In [102]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

Поделим датасет на тренировочный и тестовый в пропорции 70 на 30

In [103]:
features_0 = df[['Genre', 'Current price', 'engine', 'Number of Platforms', 'Rating']]
target = df['Amount of recommendations']

Genre и engine это категориальные признаки, поэтому используем метод get_dummies, чтобы превратить их в числовой формат с помощью one hot encoding

In [104]:
features = pd.get_dummies(features_0)

In [105]:
x_train, x_test, y_train, y_test = train_test_split(features, target, test_size=0.3, random_state=42)

In [106]:
model = LinearRegression()

In [107]:
model.fit(x_train, y_train)

LinearRegression()

In [108]:
y_pred = model.predict(x_test)

Для оценки посчитаем метрики

In [109]:
mean_squared_error(y_test, y_pred)

1058804107.5260053

Получается достаточно большая среднеквадратичная ошибка MSE, что говорит о плохом качестве модели.

In [110]:
mean_absolute_error(y_test, y_pred)

15880.652804156698

MAE тоже получается достаточно большой, в среднем модель ошибается на 15881 рекомендацию.

In [111]:
r2_score(y_test, y_pred)

-1.475814462367539

Модель плохо объясняет вариацию в данных и предсказывает результаты, которые хуже, чем простое случайное угадывание.

Попробуем улучшить линейную регрессию

Отнормируем числовые показатели от 0 до 1

In [112]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

In [113]:
num_features = ["Current price", "Number of Platforms", "Rating"]
features[num_features] = scaler.fit_transform(features[num_features])

In [114]:
x_train_sc, x_test_sc, y_train_sc, y_test_sc = train_test_split(features, target, test_size=0.3, random_state=42)

Проводим перебор по сетке для подбора наиболее оптимальных гиперпараметров и, соответственно, модели

In [115]:
grid = {'fit_intercept': [True, False], 'copy_X': [True, False], 'positive': [True, False], 'n_jobs': [-1, 0, 1, 2, 3, 4]}
model = LinearRegression()

In [116]:
search = GridSearchCV(model, grid, cv=5)
search.fit(x_train_sc, y_train_sc)
best_model = search.best_estimator_

In [117]:
y_pred_sc = best_model.predict(x_test_sc)

In [118]:
mean_squared_error(y_test_sc, y_pred_sc)

1080729081.8322742

In [119]:
mean_absolute_error(y_test_sc, y_pred_sc)

14616.036119447344

In [120]:
r2_score(y_test_sc, y_pred_sc)

-1.5270818952086649

Метрики лучше не стали, что говорит о том, что линейная регрессия не подходит для наших данных. Это может быть связано с тем, что у меня недостаточно данных для построения модели. Другой же причиной может быть выбор неподходящих признаков, что также можно свести к малому количеству данных.

In [123]:
print('https://www.youtube.com/watch?v=bqqCTC9nQDY&ab_channel=StocKMeMe')

https://www.youtube.com/watch?v=bqqCTC9nQDY&ab_channel=StocKMeMe
